In [20]:
from arcgis.gis import GIS
from arcgis.geocoding import batch_geocode, geocode
from arcgis.network.analysis import find_routes
from arcgis.mapping import WebMap
from csv import reader

In [21]:
# Defining a method
def check_if_exists(item_fl_name):
    
    search_result = gis.content.search(query = item_fl_name)

    if len(search_result) > 0:

        for item in search_result:

            item.delete()
            print('Deleted existing - ' + str(item_fl_name))

    else:

        print('No objects named - ' + item_fl_name)

In [22]:
gis = GIS("pro")

In [23]:
# Defining folder name
project_folder = 'Routes to customers'

# Checking if there's already folder with desired name and if it excists it's going to be deleted
delete_folder = gis.content.delete_folder(project_folder)

# Creating folder for feature layers and webmap
create_folder = gis.content.create_folder(folder = project_folder)

In [24]:
# Creating webmap to where feature layers will be added and what later will be published
web_map = WebMap()

In [25]:
# Defining AlphaGIS address
alphagis_address = 'Veerenni 40a-63, 10138 Tallinn'

# Geocoding AlphaGIS address
geocoding_alphagis = geocode(alphagis_address,
                             max_locations = 1,
                             as_featureset = True)

# Defining name of AlphaGIS location feature layer
alphagis_fl_name = 'AlphaGIS location'

# Checking if feature layer with desired name exists and if it excists it's going to be deleted
check_if_exists(alphagis_fl_name)

# Creating feature layer from feature set of AlphaGIS geocoded location
alphagis_fl = gis.content.import_data(geocoding_alphagis,
                                      folder = project_folder,
                                      title = alphagis_fl_name)

# Adding previously created feature layer to webmap
web_map.add_layer(alphagis_fl,
                  {'title' : alphagis_fl_name})

No objects named - AlphaGIS location


True

In [26]:
# Reading CSV file with customer addresses
with open('customer_address.csv', 'r', encoding = 'UTF-8', newline = '') as read_csv_file:
    
    csv_reader = reader(read_csv_file)
    
    # List of customer names and addresses
    rows = list(csv_reader)
    
    # Iterating trough customer list
    for item in rows:
        
        # Defining customer address
        customer_address = ' '.join(item[1:])[1:]
        
        # Geocoding customer address
        geocoding_customer = geocode(customer_address,
                                     max_locations = 1,
                                     as_featureset = True)
        
        # Defining name of customer location feature layer
        location_fl_name = item[0] + ' location'
        
        # Checking if feature layer with desired name exists and if it excists it's going to be deleted
        check_if_exists(location_fl_name)
        
        # Creating feature layer from feature set of customer geocoded location
        customer_fl = gis.content.import_data(geocoding_customer,
                                              folder = project_folder,
                                              title = location_fl_name)
        
        # Adding previously created feature layer to webmap
        web_map.add_layer(customer_fl,
                          {'title' : location_fl_name})
        
        # Geocoding customer and AlphaGIS address so that find_routes tool can be used
        batch_geocoding = batch_geocode(addresses = [alphagis_address,
                                                     customer_address],
                                        as_featureset=True)
        
        # Finding route and distance between customer and AlphaGIS
        find_route = find_routes(batch_geocoding)
        
        # Checking if find_routes tool finished successfully
        if find_route.solve_succeeded:
            print('Find_routes tool finished without errors')
        else:
            print('There was error excecuting Find_routes tool')
        
        # Defining name of route feature layer
        route_fl_name = item[0] + ' route'
        
        # Checking if feature layer with desired name exists and if it excists it's going to be deleted
        check_if_exists(route_fl_name)
        
        # Creating feature layer from feature set of route
        route_fl = gis.content.import_data(find_route.output_routes,
                                           folder = project_folder,
                                           title = route_fl_name)
        
        # Adding previously created feature layer to webmap
        web_map.add_layer(route_fl,
                          {'title' : route_fl_name})

No objects named - Kopli OÜ location
Find_routes tool finished without errors
No objects named - Kopli OÜ route
No objects named - Tehas OÜ location
Find_routes tool finished without errors
No objects named - Tehas OÜ route
No objects named - Stroomi OÜ location
Find_routes tool finished without errors
No objects named - Stroomi OÜ route
No objects named - Hippo OÜ location
Find_routes tool finished without errors
No objects named - Hippo OÜ route
No objects named - Löwen OÜ location
Find_routes tool finished without errors
No objects named - Löwen OÜ route
No objects named - Järvi OÜ location


Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Through Traffic Prohibited").


Find_routes tool finished without errors
No objects named - Järvi OÜ route
No objects named - Nurme OÜ location
Find_routes tool finished without errors
No objects named - Nurme OÜ route
No objects named - Kose OÜ location
Find_routes tool finished without errors
No objects named - Kose OÜ route
No objects named - Lume OÜ location
Find_routes tool finished without errors
No objects named - Lume OÜ route
No objects named - Waldi OÜ location
Find_routes tool finished without errors
No objects named - Waldi OÜ route


In [27]:
# Defining webmap title
web_map_title = 'Customers on map'

# Defining webmap properties
web_map_properties = {'extent' : {'xmin': 24, 'ymin': 59, 'xmax': 26, 'ymax': 60, 'spatialReference': {'wkid': 4326}},
                      'title': web_map_title,
                      'snippet':'Customer addresses and routes on map',
                      'tags':'customers'}

# Checking if webmap with desired name exists and if it excists it's going to be deleted
check_if_exists(web_map_title)

# Saving webmap
web_map_item = web_map.save(item_properties = web_map_properties,
                            folder = project_folder)

No objects named - Customers on map
